<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/ML-decision-trees-and-random-forests/decision_trees/classification/05_decision_tree_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementacja drzewa decyzyjnego

In [ ]:
import numpy as np
import pandas as pd

### Wygenerowanie zbioru danych

In [ ]:
print("#############################################")
print("Etap5. Odcinek1: Implementacja drzewa decyzyjnego. Lekkie wprowadzenie cz.1")

dataset = {'Dochód':['niski','średni','średni','średni','średni','wysoki','niski','wysoki','średni','niski'],
       'Wiek':[25, 36, 36, 25 ,28, 28, 25, 25, 36 ,36],
       'Wiarygodność':['niska','średnia','wysoka','niska','średnia','wysoka','niska','wysoka','średnia','średnia'],
       'Pożyczka':['Nie','Tak','Tak','Nie','Tak','Tak','Nie','Tak','Tak','Nie']}

#############################################
Etap5. Odcinek1: Implementacja drzewa decyzyjnego. Lekkie wprowadzenie cz.1


In [ ]:
print("Zamieniam słownik na obiekt DataFrame")
df = pd.DataFrame(dataset)
df

Zamieniam słownik na obiekt DataFrame


,Dochód,Wiek,Wiarygodność,Pożyczka
0,niski,25,niska,Nie
1,średni,36,średnia,Tak
2,średni,36,wysoka,Tak
3,średni,25,niska,Nie
4,średni,28,średnia,Tak
5,wysoki,28,wysoka,Tak
6,niski,25,niska,Nie
7,wysoki,25,wysoka,Tak
8,średni,36,średnia,Tak
9,niski,36,średnia,Nie


### Entropia:
### $$E = - \sum_{i=1}^{N}p_{i} \cdot log_{2}p_{i}$$

gdzie $p_{i}$ jest proporcją  liczby elementów w tej grupie podziału do liczby elementów w grupie przed podziałem, czyli udziałem danej klasy w poszczególnym węźle

### Łagodne wprowadzenie

In [ ]:
print("Entropia dla korzenia (root). Na początku 0, bo trzaba jakąś wartość przypisać")
entropy_node = 0
print("Wartości unikalne ze zmiennej Pożyczka")
values = df.Pożyczka.unique()
print(values)

print("Rozkład zmiennej Pożyczka")
print(df.Pożyczka.value_counts())

print("Ilość wartości Nie")
print(df.Pożyczka.value_counts()["Nie"])
print("Udział wartości Nie w całym DataFrame")
print(df.Pożyczka.value_counts()["Nie"] / len(df.Pożyczka))

print("Ilość wartości Tak")
print(df.Pożyczka.value_counts()["Tak"])
print("Udział wartości Tak w całym DataFrame")
print(df.Pożyczka.value_counts()["Tak"] / len(df.Pożyczka))

print("Liczę udziały p_i dla atrybutu Pożyczka, a potem przypisuję je do zmiennej fraction.")
for value in values:
    fraction = df.Pożyczka.value_counts()[value] / len(df.Pożyczka)
    entropy_node += -fraction * np.log2(fraction) 

print("Entropia dla zmiennej Pożyczka (naszego targetu)", entropy_node)


Entropia dla korzenia (root). Na początku 0, bo trzaba jakąś wartość przypisać
Wartości unikalne ze zmiennej Pożyczka
['Nie' 'Tak']
Rozkład zmiennej Pożyczka
Tak    6
Nie    4
Name: Pożyczka, dtype: int64
Ilość wartości Nie
4
Udział wartości Nie w całym DataFrame
0.4
Ilość wartości Tak
6
Udział wartości Tak w całym DataFrame
0.6
Liczę udziały p_i dla atrybutu Pożyczka, a potem przypisuję je do zmiennej fraction.
Entropia dla zmiennej Pożyczka (naszego targetu) 0.9709505944546686


In [ ]:
print("#############################################")
print("Etap5. Odcinek2: Implementacja drzewa decyzyjnego. Lekkie wprowadzenie cz.2")

print("Zarys.")
print("1. Obliczenie entropii dla zmiennej docelowej")
print("1.1 Obliczenie entropii dla każdego atrybutu")
print("1.2 Znalezienie atrybutu o największym zysku informacyjnym")
print("1.3 Podział względem tego atrybutu")
print("1.4 Dopóki podzbiory nie będą 'czyste' powrót do kroku 1.1")

#############################################
Etap5. Odcinek2: Implementacja drzewa decyzyjnego. Lekkie wprowadzenie cz.2
Zarys.
1. Obliczenie entropii dla zmiennej docelowej
1.1 Obliczenie entropii dla każdego atrybutu
1.2 Znalezienie atrybutu o największym zysku informacyjnym
1.3 Podział względem tego atrybutu
1.4 Dopóki podzbiory nie będą 'czyste' powrót do kroku 1.1


In [ ]:
df

,Dochód,Wiek,Wiarygodność,Pożyczka
0,niski,25,niska,Nie
1,średni,36,średnia,Tak
2,średni,36,wysoka,Tak
3,średni,25,niska,Nie
4,średni,28,średnia,Tak
5,wysoki,28,wysoka,Tak
6,niski,25,niska,Nie
7,wysoki,25,wysoka,Tak
8,średni,36,średnia,Tak
9,niski,36,średnia,Nie


In [ ]:
print("Punkt 1, czyli entropia dla zmiennej docelowej target był zrobiony wcześniej: " + str(entropy_node))
print("Robię punkt 1.1, czyli obliczam entropię dla każdej zmiennej (atrybutu) objaśniającego")

print("Przykładowo weźmy Wiarygodność")
attribute = 'Wiarygodność'
target_variables = df.Pożyczka.unique()
print("target_variables =",target_variables)
print("Do zmiennej variables przypisuję unikalne wartości atrybutu (w tym wypadku atrybut Wiarygodność)")
variables = df[attribute].unique()
print(variables)
print("Inicjalizuję zmienną przechowywującą entropię dla atrybutu (wartość początkowa niech będzie 0)")
entropy_attribute = 0
print("\nBiorę wiersze, w których atrybut Wiarygodność wynosi 'niska' oraz nie dostał pożyczki")
print(df[df[attribute] == 'niska'][df['Pożyczka'] == 'Nie'])
print("\nBiorę wiersze, w których atrybut Wiarygodność wynosi 'średnia'")
print(df[df[attribute] == 'średnia'])
print("\nBiorę wiersze, w których atrybut Wiarygodność wynosi 'średnia' oraz nie dostał pożyczki")
print(df[df[attribute] == 'średnia'][df["Pożyczka"] == "Nie"])

epsilon = np.finfo(float).eps # najmniejsza liczba typu float (po to żeby nie dzielić przez zero)
entropy_attribute = 0

def calc_entropy(attribute):
    """
    Funkcja obliczająca entropię dla wskazanego atrybutu (zmiennej).
    """
    target_variables = df.Pożyczka.unique()
    variables = df[attribute].unique()
    entropy_attribute = 0

    # Przechodzę przez wszystkie wartości atrybutu
    for variable in variables:
        entropy_each_feature = 0
        # dla każdej wartości ze zbioru "Tak" i "Nie"
        for target_variable in target_variables:
            # num to skrót od numerable, czyli licznik ułamka. Jest to liczba wierszy po wycięciu
            num = len(df[df[attribute] == variable][df.Pożyczka == target_variable]) 
            # den to denominator czyli mianownik ułamka
            den = len(df[df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy_each_feature += -(fraction * np.log2(fraction + epsilon))
        # liczę średnią ważoną entropii
        num_attr = len(df[attribute][df[attribute] == variable]) # ilość unikalnych wartości w atrybucie
        den_attr = len(df)
        fraction_attr = num_attr / den_attr
        entropy_attribute += -(fraction_attr * entropy_each_feature)
    entropy_attribute = abs(entropy_attribute)
    return entropy_attribute

Punkt 1, czyli entropia dla zmiennej docelowej target był zrobiony wcześniej: 0.9709505944546686
Robię punkt 1.1, czyli obliczam entropię dla każdej zmiennej (atrybutu) objaśniającego
Przykładowo weźmy Wiarygodność
target_variables = ['Nie' 'Tak']
Do zmiennej variables przypisuję unikalne wartości atrybutu (w tym wypadku atrybut Wiarygodność)
['niska' 'średnia' 'wysoka']
Inicjalizuję zmienną przechowywującą entropię dla atrybutu (wartość początkowa niech będzie 0)

Biorę wiersze, w których atrybut Wiarygodność wynosi 'niska' oraz nie dostał pożyczki
   Dochód  Wiek Wiarygodność Pożyczka
0   niski    25        niska      Nie
3  średni    25        niska      Nie
6   niski    25        niska      Nie

Biorę wiersze, w których atrybut Wiarygodność wynosi 'średnia'
   Dochód  Wiek Wiarygodność Pożyczka
1  średni    36      średnia      Tak
4  średni    28      średnia      Tak
8  średni    36      średnia      Tak
9   niski    36      średnia      Nie

Biorę wiersze, w których atrybut Wiar

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
entropy_Dochod = calc_entropy('Dochód')
entropy_Wiek = calc_entropy('Wiek')
entropy_Wiarygodnosc = calc_entropy('Wiarygodność')
# funkcją cacl_entropy() nie mogę liczyć entropi dla targetu bo wychodzi bez sensu wynik 3.2034265038149176e-16
# entropy_Pożyczka = calc_entropy("Pożyczka")

print('Entropia Dochód', entropy_Dochod)
print('Entropia Wiek', entropy_Wiek)
print('Entropia Wiarygodność', entropy_Wiarygodnosc)
# print('Entropia Pożyczka', entropy_Pożyczka)
print("Najczystszym zbiorem jest zbiór Wiarygodność bo ma najmniejszą entropię")

Entropia Dochód 0.3609640474436807
Entropia Wiek 0.6490224995673056
Entropia Wiarygodność 0.32451124978365264
Najczystszym zbiorem jest zbiór Wiarygodność bo ma najmniejszą entropię


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
# to było na github
print("1.2 Znalezienie atrybutu o największym zysku informacyjnym")
def information_gain(entropy_node, entropy_attribute):
    """
    Funkcja zwracająca zysk informacyjny.
    """
    # entropia przed - entropia po
    return entropy_node - entropy_attribute

ig_Dochod = information_gain(entropy_node, entropy_Dochod)
ig_Wiek = information_gain(entropy_node, entropy_Wiek)
ig_Wiarygodnosc = information_gain(entropy_node, entropy_Wiarygodnosc)

print('Zysk informacyjny {}: {}'.format('Dochód', round(ig_Dochod, 4)))
print('Zysk informacyjny {}: {}'.format('Wiek', round(ig_Wiek, 4)))
print('Zysk informacyjny {}: {}'.format('Wiarygodność', round(ig_Wiarygodnosc, 4)))

1.2 Znalezienie atrybutu o największym zysku informacyjnym
Zysk informacyjny Dochód: 0.61
Zysk informacyjny Wiek: 0.3219
Zysk informacyjny Wiarygodność: 0.6464


In [ ]:
print("#############################################")
print("Etap5. Odcinek3: Implementacja drzewa decyzyjnego - cz.1")

#############################################
Etap5. Odcinek3: Implementacja drzewa decyzyjnego - cz.1


### Implementacja prostego drzewa decyzyjnego

$\cdot$ funkcja licząca entropię zmiennej objaśnianej (targetu) 

$\cdot$ funkcja licząca entropię dla konkretnego atrybutu

$\cdot$ funkcja zwracająca atrybut z najwyższym zyskiem informacyjnym

$\cdot$ funkcja zwracająca podzbiór po podziale danego węzła

$\cdot$ funkcja budująca drzewo

In [ ]:
print("Kolumny w df")
print(df.keys())
print("Ostatnia kolumna z df")
print(df.keys()[-1])

Kolumny w df
Index(['Dochód', 'Wiek', 'Wiarygodność', 'Pożyczka'], dtype='object')
Ostatnia kolumna z df
Pożyczka


In [ ]:
print("Funkcja licząca entropię zmiennej objaśnianej (targetu).")
print("Uogólniona tak, aby ostatnia kolumna była brana jako zmienna objaśniana (target).")

def find_entropy(df):
    """
    Funkcja zwracająca entropię zmiennej objaśnianej (target).
    """
    # zwraca ostatnią kolumnę
    label = df.keys()[-1]
    # inicjaluzuję zmienną entropy
    entropy = 0
    # unikalne wartości z kolumny target (u nas "Tak" lub "Nie")
    values = df[label].unique() 
    for value in values:
        fraction = df[label].value_counts()[value] / len(df[label])
        entropy += -fraction * np.log2(fraction) 
    return entropy

Funkcja licząca entropię zmiennej objaśnianej (targetu).
Uogólniona tak, aby ostatnia kolumna była brana jako zmienna objaśniana (target).


In [ ]:
print("Wywołuję funkcję find_entropy dla mojej df")
find_entropy(df)

Wywołuję funkcję find_entropy dla mojej df


0.9709505944546686

In [ ]:
print("#############################################")
print("Etap5. Odcinek4: Implementacja drzewa decyzyjnego - cz.2")

print("Definicja funkcji liczącej entropię dla dowolnego atrybutu")
def find_entropy_attribute(df, attribute):
    """
    Funkcja zwraca entropię dla wskazanego atrybutu.
    """
    label = df.keys()[-1]
    target_variables = df[label].unique()
    variables = df[attribute].unique()
    entropy_attribute = 0

    # Przechodzę przez wszystkie wartości atrybutu
    for variable in variables:
        entropy_each_feature = 0
        # dla każdej wartości ze zbioru "Tak" i "Nie"
        for target_variable in target_variables:
            # num to skrót od numerable, czyli licznik ułamka. Jest to liczba wierszy po wycięciu
            num = len(df[df[attribute] == variable][df[label] == target_variable]) 
            # den to denominator czyli mianownik ułamka
            den = len(df[df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy_each_feature += -(fraction * np.log2(fraction + epsilon))
        # liczę średnią ważoną entropii
        num_attr = len(df[attribute][df[attribute] == variable])
        den_attr = len(df)
        fraction_attr = num_attr / den_attr
        entropy_attribute += -(fraction_attr * entropy_each_feature)
    entropy_attribute = abs(entropy_attribute)
    return entropy_attribute

#############################################
Etap5. Odcinek4: Implementacja drzewa decyzyjnego - cz.2
Definicja funkcji liczącej entropię dla dowolnego atrybutu


In [ ]:
print("Entropia dla zmiennej Dochód", find_entropy_attribute(df, "Dochód"))
print("Entropia dla zmiennej Wiek", find_entropy_attribute(df, "Wiek"))
print("Entropia dla zmiennej Wiarygodność", find_entropy_attribute(df, "Wiarygodność"))

Entropia dla zmiennej Dochód 0.3609640474436807
Entropia dla zmiennej Wiek 0.6490224995673056
Entropia dla zmiennej Wiarygodność 0.32451124978365264


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
print("Definicja funkcji zwracającej atrybut o największym zysku informacyjnym")
def find_winner(df):
    """
    Funkcja zwraca nazwę atrybutu według, którego nastąpi podział (atrybut
    posiadający największy zysk informacyjny).
    """
    # pusta lista information gain
    IG = []
    # przechodzę po nazwach wszystkich zmienych oprócz zmiennej targetowej label
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
    # zwraca atrybut o największym zysku informacyjnym
    return df.keys()[:-1][np.argmax(IG)]

Definicja funkcji zwracającej atrybut o największym zysku informacyjnym


In [ ]:
print("Znajduję atrybut o największym zysku informacyjnym")
print(find_winner(df))

Znajduję atrybut o największym zysku informacyjnym
Wiarygodność


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
print("Definicja funkcji, która zwraca podzbiór po podziale danego węzła")
def get_subtable(df, node, value):
    # zwraca część wyciętego obiektu df tam gdzie węzeł przyjmuje wartość value i resetuje index
    return df[df[node] == value].reset_index(drop=True)

Definicja funkcji, która zwraca podzbiór po podziale danego węzła


In [ ]:
print("Wywołanie funkcji get_subtable")
get_subtable(df, "Dochód", "niski")

Wywołanie funkcji get_subtable


,Dochód,Wiek,Wiarygodność,Pożyczka
0,niski,25,niska,Nie
1,niski,25,niska,Nie
2,niski,36,średnia,Nie


In [ ]:
print("#############################################")
print("Etap5. Odcinek5: Implementacja drzewa decyzyjnego - cz.3")

print("Definiuję funkcję budującą drzewo.")

print("Wstępne:")
print("Zmienna targetu")
label = df.keys()[-1]
print(label)

print()
print("Atrybut z największą wartością zysku informacyjnego")
node = find_winner(df)
print(node)

print()
print("Unikalne wartości dla atrybutu o najwyższym zysku informacyjnym")
attr_values = df[node].unique()
print(attr_values)

print()
print("Pusty słownik.")
tree ={}
print("Przypisuję atrybut z największym IG jako klucz w słowniku tree")
tree[node] = {}
print(tree)

print()
print("Wycina podtabelę")
for value in attr_values:
    subtable = get_subtable(df, node, value)
    print(node, "ma wartość", value)
    print(subtable)
    # return_counts = True aby dostać obie wartości
    # zwraca liczebność Tak i Nie
    class_value, counts = np.unique(subtable[label], return_counts=True)

    if len(counts) == 1:
        tree[node][value] = class_value[0]
    else:
        tree[node][value] = build_tree(subtable)

print()
print("Parametr tree jest słownikiem, który będzie pamiętał jak wygląda drzewo")
def build_tree(df, tree=None):
    pass

#############################################
Etap5. Odcinek5: Implementacja drzewa decyzyjnego - cz.3
Definiuję funkcję budującą drzewo.
Wstępne:
Zmienna targetu
Pożyczka

Atrybut z największą wartością zysku informacyjnego
Wiarygodność

Unikalne wartości dla atrybutu o najwyższym zysku informacyjnym
['niska' 'średnia' 'wysoka']

Pusty słownik.
Przypisuję atrybut z największym IG jako klucz w słowniku tree
{'Wiarygodność': {}}

Wycina podtabelę
Wiarygodność ma wartość niska
   Dochód  Wiek Wiarygodność Pożyczka
0   niski    25        niska      Nie
1  średni    25        niska      Nie
2   niski    25        niska      Nie
Wiarygodność ma wartość średnia
   Dochód  Wiek Wiarygodność Pożyczka
0  średni    36      średnia      Tak
1  średni    28      średnia      Tak
2  średni    36      średnia      Tak
3   niski    36      średnia      Nie
Wiarygodność ma wartość wysoka
   Dochód  Wiek Wiarygodność Pożyczka
0  średni    36       wysoka      Tak
1  wysoki    28       wysoka      Tak
2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

In [ ]:
def build_tree(df, tree=None):
    
    label = df.keys()[-1]
    
    # atrybut z największą wartością zysku informacyjnego
    node = find_winner(df)
    
    # unikalne wartości dla wskazanego atrybutu
    att_values = np.unique(df[node])
    
    # utworzenie pustego słownika do przechowania drzewa 
    if tree is None:
        tree = {}
        tree[node] = {}
    
    # wykonujemy pętlę aby zbudować drzewo, jeśli podzbiór jest czysty
    # zatrzymujemy działanie pętli
    for value in att_values:
        
        subtable = get_subtable(df, node, value)
        class_value, counts = np.unique(subtable[label], return_counts=True)
        
        if len(counts) == 1:
            tree[node][value] = class_value[0]
        else:
            tree[node][value] = build_tree(subtable)
            
    return tree

In [ ]:
tree = build_tree(df)
tree

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/li

{'Wiarygodność': {'niska': 'Nie',
  'wysoka': 'Tak',
  'średnia': {'Dochód': {'niski': 'Nie', 'średni': 'Tak'}}}}

In [ ]:
print("Ładniejszy wygląd")
import pprint
pprint.pprint(tree)
print("Jeśli wiarygodność jest niska to nie przyznaje pożyczki.")
print("Jeśli wiarygodność jest wysoka to przyznaje pożyczkę.")
print("Jeśli wiarygodność jest średnia to kolejna decyzja: patrzymy na dochód.")
print("Jeśli jest niski to nie udzielono, a jeśli jest śreni to przyznaje.")

Ładniejszy wygląd
{'Wiarygodność': {'niska': 'Nie',
                  'wysoka': 'Tak',
                  'średnia': {'Dochód': {'niski': 'Nie', 'średni': 'Tak'}}}}


## Predykcja na podstawie zbudowanego modelu¶

In [ ]:
print("#############################################")
print("Etap5. Odcinek6: Predykcja na podstawie modelu")

print()
print("Wyrywam próbkę danych z modelu. Szósty indeks")
sample = df.iloc[6]
print(sample)

#############################################
Etap5. Odcinek6: Predykcja na podstawie modelu

Wyrywam próbkę danych z modelu. Szósty indeks
Dochód          niski
Wiek               25
Wiarygodność    niska
Pożyczka          Nie
Name: 6, dtype: object


In [ ]:
tree.keys()

dict_keys(['Wiarygodność'])

In [ ]:
print()
def predict(inst, tree):
    
    for nodes in tree.keys():
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break
    return prediction

In [ ]:
predict(sample, tree)
print("Czyli zgadza się bo historycznie nie została mu udzielona pożyczka")

'Nie'